In [1]:
import xml.etree.ElementTree as ET
import pandas as pd
from tqdm.notebook import tqdm

In [ ]:
# FILENAME = "/Users/anaya/Downloads/full database.xml"
FILENAME = "/home/anaya/Projects/deep-learning-26wi/datasets/drugbank.xml"
OUT_CSV_NAME = "/home/anaya/Projects/deep-learning-26wi/datasets/interactions.csv"
OUT_TSV_NAME = "/home/anaya/Projects/deep-learning-26wi/datasets/interactions.tsv"
NAMESPACE = {"db": "http://www.drugbank.ca"}
tree = ET.parse(FILENAME)
root = tree.getroot()

In [ ]:
with open(OUT_TSV_NAME, "w") as f:
    # f.write(f"drug_primary_id,drug_name,other_drug_id,other_drug_name,interaction_description\n")
    f.write(f"drug_primary_id\tdrug_name\tother_drug_id\tother_drug_name\tinteraction_description\n")
    for drug in tqdm(root.findall("db:drug", NAMESPACE)):
        drug_name = drug.find("db:name", NAMESPACE).text
        drug_id = drug.find("db:drugbank-id[@primary='true']", NAMESPACE).text
        drug_interactions = drug.find("db:drug-interactions", NAMESPACE)
        for interaction in drug_interactions.findall("db:drug-interaction", NAMESPACE):
            other_drug_name = interaction.find("db:name", NAMESPACE).text
            other_drug_id = interaction.find("db:drugbank-id", NAMESPACE).text
            interaction_description = interaction.find("db:description", NAMESPACE).text
            # f.write(f"{drug_id},{drug_name},{other_drug_id},{other_drug_name},{interaction_description}\n")
            f.write(f"{drug_id}\t{drug_name}\t{other_drug_id}\t{other_drug_name}\t{interaction_description}\n")

  0%|          | 0/19830 [00:00<?, ?it/s]

In [15]:
i = 0
for inter in root.iter("drug-interaction"):
    print("hi")
    print(inter.attrib)
    i += 1
    if i > 10:
        break

In [7]:
import xml.etree.ElementTree as ET

def get_all_xml_tags(filename):
    """
    Reads an XML file and returns a unique list of all element tags.

    Args:
        filename (str): The path to the XML file.

    Returns:
        list: A sorted list of unique tag names.
    """
    try:
        # Parse the XML file
        tree = ET.parse(filename)
        root = tree.getroot()

        # Use a set comprehension with iter() to get all unique tags
        tags = {elem.tag for elem in root.iter()}

        # Convert to a list and sort for a consistent output
        return sorted(list(tags))
    except ET.ParseError as e:
        print(f"Error parsing XML file: {e}")
        return []
    except FileNotFoundError:
        print(f"Error: The file '{filename}' was not found.")
        return []

# Example Usage:
# Assuming you have an 'example.xml' file
# tags_list = get_all_xml_tags('example.xml')
# print(tags_list)


In [8]:
l = get_all_xml_tags(FILENAME)
[x[len("{http://www.drugbank.ca}"):] for x in l]

['absorption',
 'action',
 'actions',
 'adverse-reaction',
 'affected-organism',
 'affected-organisms',
 'ahfs-codes',
 'allele',
 'alternative-parent',
 'amino-acid-sequence',
 'approved',
 'article',
 'articles',
 'atc-code',
 'atc-codes',
 'attachment',
 'attachments',
 'average-mass',
 'calculated-properties',
 'carrier',
 'carriers',
 'cas-number',
 'categories',
 'category',
 'cellular-location',
 'chromosome-location',
 'citation',
 'class',
 'classification',
 'clearance',
 'company',
 'cost',
 'country',
 'defining-change',
 'description',
 'direct-parent',
 'dosage',
 'dosage-form',
 'dosages',
 'dpd-id',
 'drug',
 'drug-interaction',
 'drug-interactions',
 'drugbank',
 'drugbank-id',
 'drugs',
 'effect',
 'ema-ma-number',
 'ema-product-code',
 'ended-marketing-on',
 'enzyme',
 'enzymes',
 'experimental-properties',
 'expires',
 'external-identifier',
 'external-identifiers',
 'external-link',
 'external-links',
 'fda-application-number',
 'fda-label',
 'food-interaction',
 '

In [ ]:
n

In [6]:
# Source - https://stackoverflow.com/a/10077069
# Posted by K3---rnc, modified by community. See post 'Timeline' for change history
# Retrieved 2026-02-21, License - CC BY-SA 3.0

from collections import defaultdict

def etree_to_dict(t):
    d = {t.tag: {} if t.attrib else None}
    children = list(t)
    if children:
        dd = defaultdict(list)
        for dc in map(etree_to_dict, children):
            for k, v in dc.items():
                dd[k].append(v)
        d = {t.tag: {k:v[0] if len(v) == 1 else v for k, v in dd.items()}}
    if t.attrib:
        d[t.tag].update(('@' + k, v) for k, v in t.attrib.items())
    if t.text:
        text = t.text.strip()
        if children or t.attrib:
            if text:
              d[t.tag]['#text'] = text
        else:
            d[t.tag] = text
    return d


In [9]:
tree = ET.parse(FILENAME)
as_dict = etree_to_dict(tree.getroot())

In [11]:
import json

In [12]:
with open("out.json", "w") as f:
    json.dump(as_dict, f)